In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import hashlib
from datetime import datetime

from hops import hdfs

2022-05-26 21:29:14,950 INFO: generated new fontManager


### We will use rides.csv dataset for testing purposes
It doesn't have any missing data. Shape = (41078, 11).

In [6]:
df = pd.read_csv("hdfs:///Projects/{}/Resources/rides.csv".format(hdfs.project_name()), index_col=0,
                       parse_dates=["pickup_datetime"])

/srv/hops/anaconda/envs/theenv/lib/python3.8/site-packages/fsspec/implementations/hdfs.py:83: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  self.client = HadoopFileSystem(


In [8]:
import hsfs

# Create a connection
connection = hsfs.connection()

# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [17]:
df.shape

(41078, 11)

In [15]:
df.head(3)

,ride_id,is_start,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,taxi_id,driver_id
0,1228,True,1577904560000,0,-73.867676,40.759323,-73.864080,40.763897,3,2013000194,2013000194
1,527,True,1577890540000,0,-73.857086,40.772793,-73.807335,40.836117,3,2013000122,2013000122
2,394,True,1577887880000,0,-73.844320,40.789043,-73.765950,40.888790,3,2013000102,2013000102


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  bool   
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: bool(1), float64(4), int64(5), object(1)
memory usage: 3.5+ MB


#### Here we have numeric, str, bool and datetime (in UNIX format) columns

## 1) Create a feature group which does not have event time and partition key.


Connected. Call `.close()` to terminate connection gracefully.


In [21]:
df.is_start = df.is_start.astype(int)

In [23]:
df.head(5)

,ride_id,is_start,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,taxi_id,driver_id
0,1228,1,1577904560000,0,-73.867676,40.759323,-73.864080,40.763897,3,2013000194,2013000194
1,527,1,1577890540000,0,-73.857086,40.772793,-73.807335,40.836117,3,2013000122,2013000122
2,394,1,1577887880000,0,-73.844320,40.789043,-73.765950,40.888790,3,2013000102,2013000102
3,1366,1,1577907320000,0,-73.860565,40.768370,-73.819430,40.820730,3,2013000059,2013000059
4,1085,1,1577901700000,0,-73.831210,40.805737,-73.782560,40.867650,3,2013000064,2013000064


In [24]:
test1_fg = fs.create_feature_group(name="test1_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test1_fg")   
test1_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test1_fg_1_offline_fg_backfill/executions


---

## 2) Create a feature group with different primary keys

In [40]:
df["float_pk"] = np.linspace(0, 1000, num=df.shape[0])

In [41]:
df.head(3)

,ride_id,is_start,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,taxi_id,driver_id,float_pk
0,1228,1,1577904560000,0,-73.867676,40.759323,-73.864080,40.763897,3,2013000194,2013000194,0.000000
1,527,1,1577890540000,0,-73.857086,40.772793,-73.807335,40.836117,3,2013000122,2013000122,0.024345
2,394,1,1577887880000,0,-73.844320,40.789043,-73.765950,40.888790,3,2013000102,2013000102,0.048689


In [32]:
test2_float_fg = fs.create_feature_group(name="test2_float_fg",
                                   version=1,
                                   primary_key=["float_pk"],   
                                   description="test2_float_fg")   
test2_float_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test2_float_fg_1_offline_fg_backfill/executions


---

In [42]:
df["str_pk"] = df["float_pk"].astype(str)

In [43]:
df = df.drop(columns=["float_pk"])

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  int64  
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
 11  str_pk             41078 non-null  object 
dtypes: float64(4), int64(6), object(2)
memory usage: 4.1+ MB


In [45]:
test2_str_fg = fs.create_feature_group(name="test2_str_fg",
                                   version=1,
                                   primary_key=["str_pk"],   
                                   description="test2_str_fg")   
test2_str_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test2_str_fg_1_offline_fg_backfill/executions


---

In [46]:
df = df.drop(columns=["str_pk"])

In [47]:
df.is_start = df.is_start.astype(bool)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  bool   
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: bool(1), float64(4), int64(5), object(1)
memory usage: 3.5+ MB


In [49]:
test2_bool_fg = fs.create_feature_group(name="test2_bool_fg",
                                   version=1,
                                   primary_key=["is_start"],   
                                   description="test2_bool_fg")   
test2_bool_fg.save(df)

RestAPIError: Metadata operation error: (url: https://hopsworks.glassfish.service.consul:8182/hopsworks-api/api/project/123/featurestores/71/featuregroups). Server response: 
HTTP code: 500, HTTP reason: Internal Server Error, error code: 270001, error msg: Could not create feature group and corresponding Hive table, user msg: Error creating feature group table in the Hive Metastore: Invalid column type: bool

In [50]:
df.is_start = df.is_start.astype(int)

In [51]:
test2_bool_fg = fs.create_feature_group(name="test2_bool_fg",
                                   version=1,
                                   primary_key=["is_start"],   
                                   description="test2_bool_fg")   
test2_bool_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test2_bool_fg_1_offline_fg_backfill/executions


---

In [55]:
df["datetime"] = pd.to_datetime(df["pickup_datetime"], unit='ms')

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ride_id            41078 non-null  int64         
 1   is_start           41078 non-null  int64         
 2   pickup_datetime    41078 non-null  object        
 3   dropoff_datetime   41078 non-null  int64         
 4   pickup_longitude   41078 non-null  float64       
 5   pickup_latitude    41078 non-null  float64       
 6   dropoff_longitude  41078 non-null  float64       
 7   dropoff_latitude   41078 non-null  float64       
 8   passenger_count    41078 non-null  int64         
 9   taxi_id            41078 non-null  int64         
 10  driver_id          41078 non-null  int64         
 11  datetime           41078 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(6), object(1)
memory usage: 4.1+ MB


In [58]:
test2_datetime_fg = fs.create_feature_group(name="test2_datetime_fg",
                                   version=1,
                                   primary_key=["datetime"],   
                                   description="test2_datetime_fg")   
test2_datetime_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test2_datetime_fg_1_offline_fg_backfill/executions


FeatureStoreException: The Hopsworks Job failed, use the Hopsworks UI to access the job logs

---

## 3) Create feature group with ordinary datetime columns (not PK)

In [62]:
df.datetime[0]

Timestamp('2020-01-01 18:49:20')

In [61]:
test3_fg = fs.create_feature_group(name="test3_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test3_fg")   
test3_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test3_fg_1_offline_fg_backfill/executions


FeatureStoreException: The Hopsworks Job failed, use the Hopsworks UI to access the job logs

## 4) Create feature group with duplicated PK

In [66]:
df = df.drop(columns=["datetime"])

In [67]:
df.head(3)

,ride_id,is_start,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,taxi_id,driver_id
0,1228,1,1577904560000,0,-73.867676,40.759323,-73.864080,40.763897,3,2013000194,2013000194
1,527,1,1577890540000,0,-73.857086,40.772793,-73.807335,40.836117,3,2013000122,2013000122
2,394,1,1577887880000,0,-73.844320,40.789043,-73.765950,40.888790,3,2013000102,2013000102


In [68]:
df.taxi_id.nunique()

200

In [69]:
test4_fg = fs.create_feature_group(name="test4_fg",
                                   version=1,
                                   primary_key=["taxi_id"],   
                                   description="test4_fg")   
test4_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test4_fg_1_offline_fg_backfill/executions


In [70]:
import hsfs
connection = hsfs.connection()
fs = connection.get_feature_store(name='testing_featurestore')
fg = fs.get_feature_group('test4_fg', version=1)

Connected. Call `.close()` to terminate connection gracefully.


In [79]:
df_from_fg = fg.read(dataframe_type="pandas")

2022-05-25 23:01:07,518 INFO: USE `testing_featurestore`
2022-05-25 23:01:08,515 INFO: SELECT `fg0`.`ride_id` `ride_id`, `fg0`.`is_start` `is_start`, `fg0`.`pickup_datetime` `pickup_datetime`, `fg0`.`dropoff_datetime` `dropoff_datetime`, `fg0`.`pickup_longitude` `pickup_longitude`, `fg0`.`pickup_latitude` `pickup_latitude`, `fg0`.`dropoff_longitude` `dropoff_longitude`, `fg0`.`dropoff_latitude` `dropoff_latitude`, `fg0`.`passenger_count` `passenger_count`, `fg0`.`taxi_id` `taxi_id`, `fg0`.`driver_id` `driver_id`
FROM `testing_featurestore`.`test4_fg_1` `fg0`


In [81]:
df_from_fg.shape

(200, 11)

---

## 5) Create even time enabled feature group 

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  bool   
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: bool(1), float64(4), int64(5), object(1)
memory usage: 3.5+ MB


In [14]:
df = df.drop(columns=["is_start"])

In [12]:
df.pickup_datetime = df.pickup_datetime.astype(int)

In [15]:
test5_fg = fs.create_feature_group(name="test5_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test5_fg",
                                   event_time=["pickup_datetime"])   
test5_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test5_fg_1_offline_fg_backfill/executions


---

## 6) Create fg with partition_key of specific type.

In [16]:
df = pd.read_csv("hdfs:///Projects/{}/Resources/rides.csv".format(hdfs.project_name()), index_col=0,
                       parse_dates=["pickup_datetime"])

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  bool   
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: bool(1), float64(4), int64(5), object(1)
memory usage: 3.5+ MB


In [27]:
df = df.drop(columns=["is_start"])

In [24]:
df.dropoff_datetime = df.dropoff_datetime.astype(np.byte)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   pickup_datetime    41078 non-null  object 
 2   dropoff_datetime   41078 non-null  int8   
 3   pickup_longitude   41078 non-null  float64
 4   pickup_latitude    41078 non-null  float64
 5   dropoff_longitude  41078 non-null  float64
 6   dropoff_latitude   41078 non-null  float64
 7   passenger_count    41078 non-null  int64  
 8   taxi_id            41078 non-null  int64  
 9   driver_id          41078 non-null  int64  
dtypes: float64(4), int64(4), int8(1), object(1)
memory usage: 3.2+ MB


In [30]:
test6_byte_fg = fs.create_feature_group(name="test6_byte_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_byte_fg")   
test6_byte_fg.save(df)

ValueError: 0 (type <class 'int'>) do not match ['null', 'string'] on field dropoff_datetime

In [31]:
df = pd.read_csv("hdfs:///Projects/{}/Resources/rides.csv".format(hdfs.project_name()), index_col=0,
                       parse_dates=["pickup_datetime"])

In [32]:
df.is_start = df.is_start.astype(np.ubyte)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  uint8  
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: float64(4), int64(5), object(1), uint8(1)
memory usage: 3.5+ MB


In [34]:
test6_ubyte_fg = fs.create_feature_group(name="test6_ubyte_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_ubyte_fg")   
test6_ubyte_fg.save(df)

ValueError: 1 (type <class 'int'>) do not match ['null', 'string'] on field is_start

In [35]:
df.is_start = df.is_start.astype(np.short)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  int16  
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: float64(4), int16(1), int64(5), object(1)
memory usage: 3.5+ MB


In [37]:
test6_short_fg = fs.create_feature_group(name="test6_short_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_short_fg")   
test6_short_fg.save(df)

ValueError: 1 (type <class 'int'>) do not match ['null', 'string'] on field is_start

In [38]:
df.is_start = df.is_start.astype(np.ushort)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  uint16 
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: float64(4), int64(5), object(1), uint16(1)
memory usage: 3.5+ MB


In [40]:
test6_ushort_fg = fs.create_feature_group(name="test6_ushort_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_ushort_fg")   
test6_ushort_fg.save(df)

ValueError: 1 (type <class 'int'>) do not match ['null', 'string'] on field is_start

In [41]:
df.is_start = df.is_start.astype(np.intc)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  int32  
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: float64(4), int32(1), int64(5), object(1)
memory usage: 3.6+ MB


In [43]:
test6_intc_fg = fs.create_feature_group(name="test6_intc_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_intc_fg")   
test6_intc_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test6_intc_fg_1_offline_fg_backfill/executions


In [44]:
df.is_start = df.is_start.astype(np.uintc)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  uint32 
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: float64(4), int64(5), object(1), uint32(1)
memory usage: 3.6+ MB


In [46]:
test6_uintc_fg = fs.create_feature_group(name="test6_uintc_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_uintc_fg")   
test6_uintc_fg.save(df)

ValueError: 1 (type <class 'int'>) do not match ['null', 'string'] on field is_start

In [48]:
df.is_start = df.is_start.astype(np.uint)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  uint64 
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: float64(4), int64(5), object(1), uint64(1)
memory usage: 3.8+ MB


In [49]:
test6_uint_fg = fs.create_feature_group(name="test6_uint_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_uint_fg")   
test6_uint_fg.save(df)

ValueError: 1 (type <class 'int'>) do not match ['null', 'string'] on field is_start

In [50]:
df.is_start = df.is_start.astype(np.longlong)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  int64  
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: float64(4), int64(6), object(1)
memory usage: 3.8+ MB


In [51]:
test6_longlong_fg = fs.create_feature_group(name="test6_longlong_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_longlong_fg")   
test6_longlong_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test6_longlong_fg_1_offline_fg_backfill/executions


In [52]:
df.is_start = df.is_start.astype(np.ulonglong)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  uint64 
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: float64(4), int64(5), object(1), uint64(1)
memory usage: 3.8+ MB


In [53]:
test6_ulonglong_fg = fs.create_feature_group(name="test6_ulonglong_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_ulonglong_fg")   
test6_ulonglong_fg.save(df)

ValueError: 1 (type <class 'int'>) do not match ['null', 'string'] on field is_start

In [54]:
df.is_start = df.is_start.astype(np.float16)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  float16
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: float16(1), float64(4), int64(5), object(1)
memory usage: 3.5+ MB


In [55]:
test6_float16_fg = fs.create_feature_group(name="test6_float16_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_float16_fg")   
test6_float16_fg.save(df)

ValueError: 1.0 (type <class 'float'>) do not match ['null', 'string'] on field is_start

In [56]:
df.is_start = df.is_start.astype(np.single)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  float32
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: float32(1), float64(4), int64(5), object(1)
memory usage: 3.6+ MB


In [57]:
test6_single_fg = fs.create_feature_group(name="test6_single_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_single_fg")   
test6_single_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test6_single_fg_1_offline_fg_backfill/executions


In [58]:
df.is_start = df.is_start.astype(np.double)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ride_id            41078 non-null  int64  
 1   is_start           41078 non-null  float64
 2   pickup_datetime    41078 non-null  object 
 3   dropoff_datetime   41078 non-null  int64  
 4   pickup_longitude   41078 non-null  float64
 5   pickup_latitude    41078 non-null  float64
 6   dropoff_longitude  41078 non-null  float64
 7   dropoff_latitude   41078 non-null  float64
 8   passenger_count    41078 non-null  int64  
 9   taxi_id            41078 non-null  int64  
 10  driver_id          41078 non-null  int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 3.8+ MB


In [59]:
test6_double_fg = fs.create_feature_group(name="test6_double_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_double_fg")   
test6_double_fg.save(df)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test6_double_fg_1_offline_fg_backfill/executions


In [60]:
df.is_start = df.is_start.astype(np.longdouble)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   ride_id            41078 non-null  int64   
 1   is_start           41078 non-null  float128
 2   pickup_datetime    41078 non-null  object  
 3   dropoff_datetime   41078 non-null  int64   
 4   pickup_longitude   41078 non-null  float64 
 5   pickup_latitude    41078 non-null  float64 
 6   dropoff_longitude  41078 non-null  float64 
 7   dropoff_latitude   41078 non-null  float64 
 8   passenger_count    41078 non-null  int64   
 9   taxi_id            41078 non-null  int64   
 10  driver_id          41078 non-null  int64   
dtypes: float128(1), float64(4), int64(5), object(1)
memory usage: 4.1+ MB


In [61]:
test6_longdouble_fg = fs.create_feature_group(name="test6_longdouble_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_longdouble_fg")   
test6_longdouble_fg.save(df)

ArrowNotImplementedError: Unsupported numpy type 13

In [62]:
df.is_start = df.is_start.astype(np.csingle)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype    
---  ------             --------------  -----    
 0   ride_id            41078 non-null  int64    
 1   is_start           41078 non-null  complex64
 2   pickup_datetime    41078 non-null  object   
 3   dropoff_datetime   41078 non-null  int64    
 4   pickup_longitude   41078 non-null  float64  
 5   pickup_latitude    41078 non-null  float64  
 6   dropoff_longitude  41078 non-null  float64  
 7   dropoff_latitude   41078 non-null  float64  
 8   passenger_count    41078 non-null  int64    
 9   taxi_id            41078 non-null  int64    
 10  driver_id          41078 non-null  int64    
dtypes: complex64(1), float64(4), int64(5), object(1)
memory usage: 3.8+ MB


In [63]:
test6_csingle_fg = fs.create_feature_group(name="test6_csingle_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_csingle_fg")   
test6_csingle_fg.save(df)

ArrowNotImplementedError: Unsupported numpy type 14

In [64]:
df.is_start = df.is_start.astype(np.cdouble)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype     
---  ------             --------------  -----     
 0   ride_id            41078 non-null  int64     
 1   is_start           41078 non-null  complex128
 2   pickup_datetime    41078 non-null  object    
 3   dropoff_datetime   41078 non-null  int64     
 4   pickup_longitude   41078 non-null  float64   
 5   pickup_latitude    41078 non-null  float64   
 6   dropoff_longitude  41078 non-null  float64   
 7   dropoff_latitude   41078 non-null  float64   
 8   passenger_count    41078 non-null  int64     
 9   taxi_id            41078 non-null  int64     
 10  driver_id          41078 non-null  int64     
dtypes: complex128(1), float64(4), int64(5), object(1)
memory usage: 4.1+ MB


In [65]:
test6_cdouble_fg = fs.create_feature_group(name="test6_cdouble_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_cdouble_fg")   
test6_cdouble_fg.save(df)

ArrowNotImplementedError: Unsupported numpy type 15

In [66]:
df.is_start = df.is_start.astype(np.clongdouble)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype     
---  ------             --------------  -----     
 0   ride_id            41078 non-null  int64     
 1   is_start           41078 non-null  complex256
 2   pickup_datetime    41078 non-null  object    
 3   dropoff_datetime   41078 non-null  int64     
 4   pickup_longitude   41078 non-null  float64   
 5   pickup_latitude    41078 non-null  float64   
 6   dropoff_longitude  41078 non-null  float64   
 7   dropoff_latitude   41078 non-null  float64   
 8   passenger_count    41078 non-null  int64     
 9   taxi_id            41078 non-null  int64     
 10  driver_id          41078 non-null  int64     
dtypes: complex256(1), float64(4), int64(5), object(1)
memory usage: 4.7+ MB


In [67]:
test6_clongdouble_fg = fs.create_feature_group(name="test6_clongdouble_fg",
                                   version=1,
                                   primary_key=["ride_id"],   
                                   description="test6_clongdouble_fg")   
test6_clongdouble_fg.save(df)

ArrowNotImplementedError: Unsupported numpy type 16